
Etapes de scrap du site [the muse](https://www.themuse.com/).\
On utilise au maximum les json récupéré dans les appels api site pour avoir des données structurées

### Il y a 3 étapes principales. 

1.  requete sur un type de job exemple : "data engineer".
    - method: get
    - url : https://www.themuse.com/api/search-renderer/jobs?
    - params : ctsEnabled=false&query=Data+Engineer&preference=krcbqorfvz&limit=20&timeout=5000
    - recupération du json qui présente les différentes offres. Données conservées
        - job_title
        - company.short_name
        - short_title
        - posted_at
        - cursor (le dernier cursor est utile pour la pagination) --> start_after = dernier cursor
        - has_more (utile pour la pagination)

2.  récupération de chaque job dans le json reçu et requete pour obtenir le html de chaque job
    - method: get
    - url : https://www.themuse.com/jobs/
    - params: [hit.company.short_name]/[hit.short_title]

3.  dans le html, recupérér le json
    - dans la balise <script id="__NEXT_DATA__" type="application/json"></script>

### données conservées

Pour l'instant on conservce les données suivantes :


In [1]:
import aiohttp
from string import Template

site_url: str = "https://www.themuse.com"
search_url: str ="/api/search-renderer/jobs"
job_url: str = Template("/jobs/$company/$job_title")


In [53]:
from typing import Dict, List
import re

async def fetch_list_jobs(query: str, limit: int = 20, next: str | None = None)-> List[Dict]:
    """ Effectue une requête pour lister les jobs.
        Dans le cas d'une pagination, on ajoute start_after= cursor du dernier job

    Args:
        query (str): requête dont les espaces sont remplacés par des +
        limit (int, optional): Limite de réponse max. Defaults à 20.
        next (str | None, optional): cursor à partir duquel est lancé la requet si pagination. Defaults to None.

    Returns:
        List[Dict]: liste des jobs
    """
    query:str = re.sub(r'\s+','+', query)
    search_params: Dict = {
                        "ctsEnabled":"false",
                        "query":query,
                        "preference":"krcbqorfvz",
                        "limit":limit,
                        "timeout":5000
                    }
    if next :
        search_params.update({"start_after":next})

    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(f"{site_url}{search_url}", params=search_params) as resp:
                if resp.status == 200:
                    return await resp.json()
        except Exception as exc :
            print(exc)




In [54]:
# extraction des données à conserver dans les jobs
# on peut utiliser glom pour l'extraction des json
from typing import Tuple
from glom import glom

async def extract_resumed_jobs(response_api: Dict)-> Tuple[bool, List[ResumedJob]]: # type: ignore
    """_summary_

    Args:
        response_api (Dict): _description_

    Returns:
        List[ResumedJob]: _description_
    """
    job_specs = {
        "title":"hit.title",
        "short_title": "hit.short_title",
        "company_name": "hit.company.short_name",
        "score": "score",
        "cursor": "cursor",
        "posted_at": "hit.posted_at",
    }

    resumed_jobs: List[Dict] = [glom(data,  job_specs) for data in response_api.get('hits')]
    
    return (response_api.get('has_more'), sorted(resumed_jobs, key=lambda k:k.score, reverse=False ))


In [57]:


# appel de la methode une 1ere fois
resp: List[Dict] = await fetch_list_jobs("Data engineer", limit=5)
extract_jobs: Tuple[bool, List[ResumedJob]] = await extract_resumed_jobs(resp)

resumed_jobs: List[ResumedJob] = extract_jobs[1]
resumed_jobs

[ResumedJob(title='Data Engineer', short_title='data-engineer-85af17', company_name='appfire', score=91.36105, cursor='91.36105,1726551876000,2f06a4ce-6992-4de1-8b9a-3359ad458e45', posted_at=1726551876),
 ResumedJob(title='Data Engineer', short_title='data-engineer', company_name='constellationbrands', score=91.961586, cursor='91.961586,1727109629000,02ce7182-bbe9-45b3-8e12-b485ed308c09', posted_at=1727109629),
 ResumedJob(title='Data Engineer', short_title='data-engineer-d8bdf7', company_name='leidos', score=92.13994, cursor='92.13994,1727280846000,1d5c7933-ebfe-4f5a-aa7c-c4ac76da24ca', posted_at=1727280846),
 ResumedJob(title='Data Engineer', short_title='data-engineer-e67071', company_name='atlassian', score=92.154686, cursor='92.154686,1727915038000,bd22fc4b-7c06-46ee-8c3a-dfe9914eb271', posted_at=1727915038),
 ResumedJob(title='Data Engineer', short_title='data-engineer-950ad7', company_name='arcadia', score=92.484505, cursor='92.484505,1718201808000,bd345d35-45d8-48bc-84e5-aeed22

In [ ]:
# creation des ENtités qui structurent les données et permettent une conservation en base
from dataclasses import dataclass

@dataclass
class ResumedJob:
    title: str
    short_title: str
    company_name: str
    score: int # pour le tri
    cursor: str
    posted_at: int # timestamp

resumed_jobs = [ResumedJob(**job) for job in resumed_jobs]